# Hidden Markov Model 隐马尔可夫模型

隐马尔可夫模型，用来描述一个含有隐含未知参数的马尔可夫过程。

[如何用简单易懂的例子解释隐马尔可夫模型？](https://www.zhihu.com/question/20962240)

在正常的马尔可夫模型中，状态对于观察者来说是直接可见的。这样状态的转换概率便是全部的参数。而在隐马尔可夫模型中，状态并不是直接可见的，但受状态影响的某些变量则是可见的。每一个状态在可能输出的符号上都有一概率分布。因此输出符号的序列能够透露出状态序列的一些信息。

## 马尔可夫过程

马尔可夫过程是一个具备了马尔可夫性质的随机过程。具备离散状态的马尔可夫过程，通常被称为**马尔可夫链**。马尔可夫链通常使用离散的时间集合定义，又称离散时间马尔可夫链

### 马尔可夫链

假设状态序列为 $\cdots x_{t-2}, x_{t-1}, x_{t}, x_{t+1}, x_{t+2}, \cdots$，由马尔科夫链定义可知，时刻 $x_{t+1}$ 的状态只与 $x_t$ 有关，用数学公式来描述就是：

$$P\left(x_{t+1} | \cdots, x_{t-2}, x_{t-1}, x_{t}\right)=P\left(x_{t+1} | x_{t}\right)$$

### 示例

这些圆圈中的状态代表假设的股票市场是否在某个特定周内表现出牛市（ bull market），熊市（bear market）或停滞的（ stagnant market）市场走势。 根据这个数字，一个牛市周之后下一周还是牛市周的概率为 90％，是熊市概率为 7.5％，持平的概率为 2.5%。

![](./img/马尔科夫链.png)

若我们标记状态空间 {1 = bull，2 = bear，3 = stagnant}，则这个例子的转换矩阵是：

$$P=\left[\begin{array}{ccc}
0.9 & 0.075 & 0.025 \\
0.15 & 0.8 & 0.05 \\
0.25 & 0.25 & 0.5
\end{array}\right]$$

## 隐马尔可夫模型

![](./img/hmm.png)

### Python 示例

In [1]:
import numpy as np
import pandas as pd

In [2]:
class MyHMM:
    def __init__(self, p, A, B, Ob, method="forward"):
        self.p = p  # 初始状态概率
        self.A = A  # 状态转移概率分布
        self.B = B  # 观测概率分布
        self.Ob = Ob  # 观测序列
        self.method = method  # 采用算法

    def forward(self):
        """
        前向算法
        """
        alpha = self.p*self.B[self.Ob[0]]
        for i in range(1, self.Ob.size):
            alpha = alpha.dot(self.A)*self.B[self.Ob[i]]
        return alpha.sum()

    def back(self):
        """
        后向算法
        """
        beta = np.ones(self.p.size)
        for i in range(1, self.Ob.size):
            beta = self.A.dot(self.B[self.Ob[self.Ob.size-i]]*beta)
        res = self.p.dot(self.B[self.Ob[0]]*beta)
        return res

    def fit(self):
        if "forward" in self.method:
            print("使用前向算法")
            res = self.forward()
        elif "back" in self.method:
            print("使用后向算法")
            res = self.back()
        else:
            raise ValueError
        return res

    def best_way(self):
        """
        最优路径
        """
        index = []
        sigema = self.p*self.B[self.Ob[0]]
        for i in range(1, self.Ob.size):
            _sigema = sigema.values*(self.A.T)
            col_index = np.argmax(_sigema, axis=1)
            row_index = np.arange(self.p.size)
            sigema = _sigema[row_index, col_index]*self.B[self.Ob[i]]
            index.append(dict(enumerate(col_index)))
        best_index = []
        k = np.argmax(np.array(sigema))
        best_index.append(k)
        for i in range(len(index)):
            k = index[len(index)-1-i][k]
            best_index.append(k)
        best_index = np.array(best_index[::-1])+1
        return best_index

In [3]:
A = np.array([[0.5, 0.2, 0.3],
              [0.3, 0.5, 0.2],
              [0.2, 0.3, 0.5]])

B = pd.DataFrame(np.array([[0.5, 0.5],
                           [0.4, 0.6],
                           [0.7, 0.3]]), columns=["红", "白"])

p = np.array([0.2, 0.4, 0.4])

Ob = np.array(["红", "白", "红", "白"])

clf = MyHMM(p, A, B, Ob, method="back")
res = clf.fit()
print(round(res, 2))

使用后向算法
0.06


In [4]:
best_way = clf.best_way()
print(best_way)

[3 2 2 2]
